# Week #8 - The Spectral element method

### Introduction

- It is a mixing combining the best of each method that we have discussed so far.
- Heterogeneous grid
- Curved grid elements (geometrical flexibility)

For time evolution we use the usual time extrapolation scheme
(finite difference method), the Garlekin principal for the linear interpolation.
Inside the element we are going to use concepts of the *Pseudospectral method*.
That will allow us to set the Mass matrix (which needs to be inverted) into
a diagonal form. At first glance we didn't have problems to diagonalize it.
However, when one handles three dimensional data involving a huge amount amount
of data, that becomes an issue and inverting the matrix is no longer possible.
Consequently, that is an essential step towards solving the problem.

The next steps are:
- Development of the weak form
- Lagrange interpolation
- Numerical integration


### Weak Form - Matrix Formulation


As we have seen before, we start from the elastic wave equation, multiply
it by a test function, $phi$ and integrate the one containing $\partial_x$
by parts to obtain the weak form. In geophysics it is rather common to have
free source boundary condition, there this term vanishes and we readly obtain

$$
    \int_D\mathrm{d}x\rho(x)\partial_t^2u\phi + 
    \int_D\mathrm{d}x\mu(x)\partial_xu\partial_x\phi = 
    \int_D\mathrm{d}xf\phi\,.
$$

Using the discrete version of our amplitude $u$ and its decomposition 
in the basis functions $phi_i$, we have

$$
    u(x,t) = \sum_{i=1}^{N_p}u_i(t)\phi_i(x)\,.
$$

Replacing it into the previous we equation we have a lengthy,
but known formula, given by

$$
    \sum_{i=1}^{N_p}\partial_t^2u_i(t)\int_D\mathrm{d}x\rho(x)\phi_j(x)\phi_i(x) + 
    \sum_{i=1}^{N_p}u_i(t)\int_D\mathrm{d}x\mu(x)\partial_x\partial_x\phi_i(x)\partial_x\phi_j(x) = 
    \int_D\mathrm{d}xf\phi_i(x)\,,
$$
where it can be cast into a matrix form, similar to what we have already done

$$
    \mathbf{M}\cdot\partial_t^2\mathbf{u}(t) + \mathbf{K}\cdot\mathbf{u}(t) =
    \mathbf{f}(t)\,.
$$

Using the finite difference method for the time derivative, this equation can be inverted
obtaining

$$
    \mathbf{u}^\mathrm{new} =
        dt^2\mathbf{M}^{-1}\cdot\Big(\mathbf{f} - \mathbf{K}\cdot\mathbf{u}\Big) +
        2\mathbf{u} - \mathbf{u}^\mathrm{old}
$$

The integral above encompass the entire domain. The next step is to transform
it to be performed within a single element.

### Element Level

Supposing that the domain $D$ is partioned into N elements, consequently
having $N+1$ points, and we want make a transformation to work in a single
element. We want to work on a new domain (inspired by the definition of the
Lagrange functions) defined in $-1\leq\xi\leq 1$. If we set the zero of our
new coordinate system at the middle of the element we can define the left
boundary of our element to be $\xi=-1$ and the right boundary to be $\xi=1$.
Suppose that we are analyzing the element which starts at the position $x_i$
and finishes at the position $x_{i+1}$, such that $x_{i+1} - x_i = h_i$.
We now want that when $\xi = -1 \rightarrow x \equiv x_{i}$ and when
$\xi = +1 \rightarrow x \equiv x_{i+1} = x_{i}+h_{i}$.

It is easy to see that the transformation which allows that is the following

$$
    x \equiv x(\xi) = x_i + h_i\frac{\xi+1}{2}\,.
$$
and we can invert this equation to obtain

$$
    \xi = 2\frac{x - x_i}{h_i} - 1\,.
$$

The integrals that we needed to calculate over the entire $x-$ domain
now are performed within individual elements and it is given by

$$
    \int_D\mathrm{d}x G(x) \rightarrow \int_{-1}^1 \mathrm{d}\xi\frac{dx}{d\xi}G(\xi)
        = \frac{1}{2}\int_{-1}^1\mathrm{d}\xi h_iG(\xi)
$$

### Lagrange Interpolation

The Lagrange polynomials spam the basis functions that we are going
to use. They are defined as follows

$$
    \phi_i \equiv \ell^{(N)}_i(\xi) = \prod_{j\neq i =1}^{N+1}\frac{\xi-\xi_i}{\xi_i - \xi_j}\,.
$$

At the collocation points the Lagrange polynomials have the delta-Kronecker property,
$i.e.,$

$$
    \ell^{(N)}_i(\xi_j) = \delta_{ij}\,.
$$

The selection about the collocation points can be done by several
different ways. One of them is the *Gauss-Lobatto-Legendre* points.
This choice leads us to a distribution similar to the one of the
Chebyschev polynomials. The derivative are zero at the collocation
points.

When one starts increasing the order of the polynomials, the region
close to the boundaries starts becoming denser and denser. It implies
that there will appear smaller and smaller lengths $dx$. Due to the
Courant criteria, one should find a balance between the precision
desired and these $dx$ lengths, because to avoid dispersive effects,
one would need, consequently, needs even smaller time steps $dt$.

Generally, for real-world simulation, a reasonable choice for the
number of collocation points is of the order of 5, leading us to
four distances.

> The difference between the Gauss-Lobatto-Legendre and the Gauss-quadrature rule, is that whereas the latter excludes the boundaries the former **include** the boundaries, since we need it for matrix elements.



In [13]:
import numpy as np

def f(x):
    return x**2/2 - x**5/3
points = [-1, -np.sqrt(3/7), 0, np.sqrt(3/7), 1]
weights = np.array([1./10, 49./90, 32./45, 49./90, 1./10])

func_at_points = list(map(f, points))

np.dot(func_at_points, weights)

#print (func_at_points)
#print (weights)

0.33333333333333326